In [16]:
import numpy as np

def calculate_iqs_unphased(true_genotypes, imputed_dosages):
    # Convert imputed dosages to discrete values
    imputed_discrete = np.round(imputed_dosages).astype(int)

    # Clip the imputed discrete values to be within the range of 0 to 2
    imputed_discrete = np.clip(imputed_discrete, 0, 2)

    # Create a contingency table
    contingency_table = np.zeros((3, 3), dtype=int)

    # Fill the contingency table
    for true_geno, imputed_geno in zip(true_genotypes, imputed_discrete):
        for true_allele, imputed_allele in zip(true_geno, imputed_geno):
            contingency_table[int(true_allele), int(imputed_allele)] += 1

    # Calculate the total number of genotypes
    total_genotypes = np.sum(contingency_table)

    # Calculate observed proportion of agreement (Po)
    observed_agreement = np.trace(contingency_table) / total_genotypes

    # Calculate marginal sums
    row_marginals = np.sum(contingency_table, axis=1)
    col_marginals = np.sum(contingency_table, axis=0)

    # Calculate chance agreement (Pc)
    chance_agreement = np.sum((row_marginals * col_marginals) / (total_genotypes ** 2))

    # Calculate IQS
    if chance_agreement == 1:  # To prevent division by zero in case of perfect chance agreement
        iqs_score = 0
    else:
        iqs_score = (observed_agreement - chance_agreement) / (1 - chance_agreement)

    return iqs_score

# Example usage:
true_genotypes = np.array([[0, 1, 2], [1, 2, 0], [2, 0, 1]])
imputed_dosages = np.array([[0.1, 1.2, 1.9], [1.0, 1.8, 0.3], [2.0, 0.5, 1.4]])

iqs_score = calculate_iqs_unphased(true_genotypes, imputed_dosages)
print(f"IQS Score: {iqs_score}")


IQS Score: 1.0


# Linear Regression Model

In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score as sklearn_r2_score, accuracy_score
import optuna
from matplotlib import pyplot as plt
import os
import csv

# Load and preprocess the data
data_directory = '../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []

# Create folders for saving files
output_folder = "../../Data/model_results_unphased_all_PRS/linear_regression/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

os.makedirs(model_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

for chromosome_number in range(start, 23):
    # Create subfolders for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    os.makedirs(chr_model_folder, exist_ok=True)
    os.makedirs(chr_csv_folder, exist_ok=True)

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*PRS313_)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='PRS313_').values, dtype=torch.float32)

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the linear regression model with L1 regularization
    class LinearRegression(nn.Module):
        def __init__(self, input_dim, output_dim, l1_coef=0.0):
            super(LinearRegression, self).__init__()
            self.linear = nn.Linear(input_dim, output_dim)
            self.l1_coef = l1_coef

        def forward(self, x):
            out = self.linear(x)
            return out

        def l1_loss(self):
            return self.l1_coef * torch.norm(self.linear.weight, p=1)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters for tuning
    input_dim = X_train_val.shape[1]
    output_dim = y_train_val.shape[1]
    num_epochs = 500

    # Define the objective function for Optuna with cross-validation and early stopping
    def objective(trial):
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
        l1_coef = trial.suggest_float('l1_coef', 1e-10, 1e-1, log=True)
        patience = trial.suggest_int('patience', 5, 20)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        lr_factor = trial.suggest_float('lr_factor', 0.1, 0.9)
        num_epochs = trial.suggest_int('num_epochs', 100, 500)

        model = LinearRegression(input_dim, output_dim, l1_coef).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_factor, patience=patience, verbose=False)

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        fold_losses = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
            X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
            y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

            train_dataset = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
            best_val_loss = float('inf')
            counter = 0

            for epoch in range(num_epochs):
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y) + model.l1_loss()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()

                train_loss /= len(train_loader)

                val_dataset = TensorDataset(X_val, y_val)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)

                with torch.no_grad():
                    val_loss = 0.0
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y) + model.l1_loss()
                        val_loss += loss.item()

                    val_loss /= len(val_loader)
                    scheduler.step(val_loss)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        counter = 0
                    else:
                        counter += 1

                    if counter >= patience:
                        break

            fold_losses.append(best_val_loss)

        return np.mean(fold_losses)
    # Create the "optuna_studies" folder if it doesn't exist
    os.makedirs("optuna_studies", exist_ok=True)

    # Create an Optuna study and optimize the hyperparameters
    study_name = f"unphased_full_23andMe_chr{chromosome_number}_study_linear_regression"
    storage_name = f"sqlite:///optuna_studies/{study_name}.db"

    # Check if the study exists
    current_dir = os.getcwd()
    study_exists = os.path.exists(current_dir + f"/optuna_studies/{study_name}.db")

    if study_exists:
        # Load the existing study
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    else:
        # Create a new study
        study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)

    study.optimize(objective, n_trials=1, n_jobs=-1)

    # Print the best hyperparameters and best value
    print(f"Chr {chromosome_number} - Best hyperparameters: {study.best_params}")
    print(f"Chr {chromosome_number} - Best value: {study.best_value:.4f}")

    # Train the final model with the best hyperparameters and early stopping
    best_learning_rate = study.best_params['learning_rate']
    best_l1_coef = study.best_params['l1_coef']
    best_patience = study.best_params['patience']
    best_batch_size = study.best_params['batch_size']
    best_lr_factor = study.best_params['lr_factor']
    best_num_epochs = study.best_params['num_epochs']

    model = LinearRegression(input_dim, output_dim, best_l1_coef).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_learning_rate)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=best_lr_factor, patience=best_patience, verbose=False)

    train_dataset = TensorDataset(X_train_val, y_train_val)
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

    best_train_loss = float('inf')
    counter = 0

    for epoch in range(best_num_epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs, batch_y) + model.l1_loss()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            counter = 0
        else:
            counter += 1

        if counter >= best_patience:
            break

        scheduler.step(train_loss)

    # Save the final model
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Final model saved at: {model_save_path}")

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_accuracy = ((y_test == test_outputs.round()).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

        # Append performance metrics to the lists
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)
        accuracy_scores.append(test_accuracy)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_unphased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='PRS').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

    # Create a DataFrame to store the performance metrics for each chromosome
    performance_df = pd.DataFrame({
        'Chromosome': list(range(start, chromosome_number + 1)),
        'R2 Score': r2_scores,
        'IQS Score': iqs_scores,
        'Accuracy Score': accuracy_scores
    })

    # Save the performance metrics to a CSV file
    performance_csv_file = csv_folder + 'performance_metrics.csv'
    performance_df.to_csv(performance_csv_file, index=False)
    print(f"Performance metrics saved at: {performance_csv_file}")

Total SNPs:  929
PRS313 SNPs:  30
Total SNPs used for Training:  899


[I 2024-05-23 14:21:51,261] Trial 86 finished with value: 0.10622426457703113 and parameters: {'learning_rate': 0.005190997400957485, 'l1_coef': 4.3763132874855275e-08, 'patience': 18, 'batch_size': 128, 'lr_factor': 0.33527929693949376, 'num_epochs': 435}. Best is trial 49 with value: 0.10217271894216537.


Chr 1 - Best hyperparameters: {'learning_rate': 0.004525680413232911, 'l1_coef': 1.4557748222420537e-06, 'patience': 20, 'batch_size': 256, 'lr_factor': 0.2721642150080666, 'num_epochs': 367}
Chr 1 - Best value: 0.1022
Final model saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/models_unphased/chr1/final_model_chr1.pth
INPUTS [[0. 2. 0. ... 2. 1. 0.]
 [0. 2. 1. ... 2. 0. 0.]
 [0. 1. 0. ... 2. 0. 0.]
 ...
 [0. 2. 0. ... 2. 1. 0.]
 [0. 1. 0. ... 2. 1. 0.]
 [0. 0. 0. ... 2. 1. 0.]] [[ 0.21295525  1.1346838   0.02625206 ...  2.3359337  -0.04398361
   0.04737796]
 [ 0.84804386  2.4524505   1.5537536  ...  2.359358    0.11665091
  -0.02905133]
 [ 0.41301873  1.6406057  -0.12913103 ...  1.702893   -0.39570338
  -0.00645405]
 ...
 [ 0.3019755   1.7119876   0.0105901  ...  2.058869    0.85444266
   0.0405995 ]
 [-0.2700336   1.7962674  -0.04037734 ...  1.9678326   1.0011257
   0.24637793]
 [ 0.36895612  1.7164937  -0.05849757 ...  1.4100678   0.7813533
   0.22586088]]
ROUN

KeyboardInterrupt: 

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as sklearn_r2_score
import os
import csv

# Load and preprocess the data
data_directory = '../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []

# Folders for saved models and CSV files
output_folder = "../../Data/model_results_unphased_all_PRS/linear_regression/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

for chromosome_number in range(start, 23):
    # Paths for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*PRS313_)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='PRS313_').values, dtype=torch.float32)

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the linear regression model with L1 regularization
    class LinearRegression(nn.Module):
        def __init__(self, input_dim, output_dim, l1_coef=0.0):
            super(LinearRegression, self).__init__()
            self.linear = nn.Linear(input_dim, output_dim)
            self.l1_coef = l1_coef

        def forward(self, x):
            out = self.linear(x)
            return out

        def l1_loss(self):
            return self.l1_coef * torch.norm(self.linear.weight, p=1)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the final model
    model = LinearRegression(X_train_val.shape[1], y_train_val.shape[1])
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model.to(device)
    model.eval()

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_accuracy = ((y_test == test_outputs.round()).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

        # Append performance metrics to the lists
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)
        accuracy_scores.append(test_accuracy)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_unphased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='PRS').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

# Create a DataFrame to store the performance metrics for each chromosome
performance_df = pd.DataFrame({
    'Chromosome': list(range(start, 23)),
    'R2 Score': r2_scores,
    'IQS Score': iqs_scores,
    'Accuracy Score': accuracy_scores
})

# Save the performance metrics to a CSV file
performance_csv_file = csv_folder + 'performance_metrics.csv'
performance_df.to_csv(performance_csv_file, index=False)
print(f"Performance metrics saved at: {performance_csv_file}")


Total SNPs:  929
PRS313 SNPs:  30
Total SNPs used for Training:  899
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr1/individual_r2_scores_chr1.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr1/individual_iqs_scores_chr1.csv
Total SNPs:  629
PRS313 SNPs:  21
Total SNPs used for Training:  608
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr2/individual_r2_scores_chr2.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr2/individual_iqs_scores_chr2.csv
Total SNPs:  963
PRS313 SNPs:  16
Total SNPs used for Training:  947
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr3/individual_r2_scores_chr3.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/linear_regression/csv_files/chr3

# Multinomial Logistic Regression Model

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score as sklearn_r2_score, accuracy_score
import optuna
from matplotlib import pyplot as plt
import os
import csv

# Load and preprocess the data
data_directory = '../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []

# Create folders for saving files
output_folder = "../../Data/model_results_unphased_all_PRS/logistic_regression/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

os.makedirs(model_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

for chromosome_number in range(start, 23):
    # Create subfolders for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    os.makedirs(chr_model_folder, exist_ok=True)
    os.makedirs(chr_csv_folder, exist_ok=True)

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*PRS313_)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='PRS313_').values, dtype=torch.long)

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the multinomial logistic regression model with L1 regularization
    class MulticlassLogisticRegression(nn.Module):
        def __init__(self, input_dim, output_dim, num_classes=3, l1_coef=0.0):
            super(MulticlassLogisticRegression, self).__init__()
            self.num_classes = num_classes
            self.linear = nn.ModuleList([nn.Linear(input_dim, output_dim) for _ in range(num_classes)])
            self.l1_coef = l1_coef

        def forward(self, x):
            out = torch.stack([linear(x) for linear in self.linear], dim=-1)
            out = nn.functional.softmax(out, dim=-1)
            return out

        def l1_loss(self):
            return self.l1_coef * sum(torch.norm(linear.weight, p=1) for linear in self.linear)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters for tuning
    input_dim = X_train_val.shape[1]
    output_dim = y_train_val.shape[1]
    num_epochs = 500

    # Define the objective function for Optuna with cross-validation and early stopping
    def objective(trial):
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
        l1_coef = trial.suggest_float('l1_coef', 1e-10, 1e-1, log=True)
        patience = trial.suggest_int('patience', 5, 20)
        batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        lr_factor = trial.suggest_float('lr_factor', 0.1, 0.9)
        num_epochs = trial.suggest_int('num_epochs', 100, 500)

        model = MulticlassLogisticRegression(input_dim, output_dim, l1_coef=l1_coef).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_factor, patience=patience, verbose=False)

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        fold_losses = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
            X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
            y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

            train_dataset = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
            best_val_loss = float('inf')
            counter = 0

            for epoch in range(num_epochs):
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                    outputs = model(batch_X)
                    loss = criterion(outputs.transpose(1, 2), batch_y) + model.l1_loss()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()

                train_loss /= len(train_loader)

                val_dataset = TensorDataset(X_val, y_val)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)

                with torch.no_grad():
                    val_loss = 0.0
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs.transpose(1, 2), batch_y) + model.l1_loss()
                        val_loss += loss.item()

                    val_loss /= len(val_loader)
                    scheduler.step(val_loss)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        counter = 0
                    else:
                        counter += 1

                    if counter >= patience:
                        break

            fold_losses.append(best_val_loss)

        return np.mean(fold_losses)
    
    # Create the "optuna_studies" folder if it doesn't exist
    os.makedirs("optuna_studies", exist_ok=True)

    # Create an Optuna study and optimize the hyperparameters
    study_name = f"unphased_full_23andMe_chr{chromosome_number}_study_logistic_regression"
    storage_name = f"sqlite:///optuna_studies/{study_name}.db"

    # Check if the study exists
    current_dir = os.getcwd()
    study_exists = os.path.exists(current_dir + f"/optuna_studies/{study_name}.db")

    if study_exists:
        # Load the existing study
        study = optuna.load_study(study_name=study_name, storage=storage_name)
    else:
        # Create a new study
        study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)

    study.optimize(objective, n_trials=50, n_jobs=-1)

    # Print the best hyperparameters and best value
    print(f"Chr {chromosome_number} - Best hyperparameters: {study.best_params}")
    print(f"Chr {chromosome_number} - Best value: {study.best_value:.4f}")

    # Train the final model with the best hyperparameters and early stopping
    best_learning_rate = study.best_params['learning_rate']
    best_l1_coef = study.best_params['l1_coef']
    best_patience = study.best_params['patience']
    best_batch_size = study.best_params['batch_size']
    best_lr_factor = study.best_params['lr_factor']
    best_num_epochs = study.best_params['num_epochs']

    model = MulticlassLogisticRegression(input_dim, output_dim, l1_coef=best_l1_coef).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=best_lr_factor, patience=best_patience, verbose=False)

    train_dataset = TensorDataset(X_train_val, y_train_val)
    train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)

    best_train_loss = float('inf')
    counter = 0

    for epoch in range(best_num_epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs.transpose(1, 2), batch_y) + model.l1_loss()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            counter = 0
        else:
            counter += 1

        if counter >= best_patience:
            break

        scheduler.step(train_loss)

    # Save the final model
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Final model saved at: {model_save_path}")

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_outputs = test_outputs.argmax(dim=-1)
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())

        test_accuracy = ((y_test == test_outputs).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

        # Append performance metrics to the lists
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)
        accuracy_scores.append(test_accuracy)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_unphased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='PRS').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

    # Create a DataFrame to store the performance metrics for each chromosome
    performance_df = pd.DataFrame({
        'Chromosome': list(range(start, chromosome_number + 1)),
        'R2 Score': r2_scores,
        'IQS Score': iqs_scores,
        'Accuracy Score': accuracy_scores
    })

    # Save the performance metrics to a CSV file
    performance_csv_file = csv_folder + 'performance_metrics.csv'
    performance_df.to_csv(performance_csv_file, index=False)
    print(f"Performance metrics saved at: {performance_csv_file}")

Total SNPs:  929
PRS313 SNPs:  30
Total SNPs used for Training:  899


[I 2024-05-23 00:33:59,250] Trial 22 finished with value: 0.733684065937996 and parameters: {'learning_rate': 0.002220891376392196, 'l1_coef': 2.7694209571735434e-08, 'patience': 16, 'batch_size': 128, 'lr_factor': 0.3090166357776345, 'num_epochs': 359}. Best is trial 12 with value: 0.6769969142400301.
[I 2024-05-23 00:34:00,673] Trial 17 finished with value: 0.745872899889946 and parameters: {'learning_rate': 0.002282542929228127, 'l1_coef': 1.7678189359217486e-08, 'patience': 16, 'batch_size': 128, 'lr_factor': 0.2898999112507365, 'num_epochs': 365}. Best is trial 12 with value: 0.6769969142400301.
[I 2024-05-23 00:34:04,619] Trial 20 finished with value: 0.7228652864694596 and parameters: {'learning_rate': 0.00217582920731858, 'l1_coef': 2.533053034976939e-08, 'patience': 16, 'batch_size': 128, 'lr_factor': 0.3031199406133188, 'num_epochs': 355}. Best is trial 12 with value: 0.6769969142400301.
[I 2024-05-23 00:34:11,111] Trial 21 finished with value: 0.7277507990598678 and paramete

Chr 1 - Best hyperparameters: {'learning_rate': 0.0003659720828647503, 'l1_coef': 2.2917141824697583e-08, 'patience': 18, 'batch_size': 32, 'lr_factor': 0.3973506534926021, 'num_epochs': 386}
Chr 1 - Best value: 0.6770
Final model saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/models_unphased/chr1/final_model_chr1.pth
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr1/individual_r2_scores_chr1.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr1/individual_iqs_scores_chr1.csv
Performance metrics saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/performance_metrics.csv
Total SNPs:  629
PRS313 SNPs:  21
Total SNPs used for Training:  608


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_98377/4219423060.py:25: RuntimeWarning: invalid value encountered in divide
  chi_squared = np.sum((contingency_table - expected_counts) ** 2 / expected_counts)
[I 2024-05-23 01:06:23,934] Trial 12 finished with value: 1.6472469466073172 and parameters: {'learning_rate': 0.07619744115418815, 'l1_coef': 0.0025746994774138057, 'patience': 12, 'batch_size': 64, 'lr_factor': 0.8290381863688072, 'num_epochs': 130}. Best is trial 2 with value: 0.7424067212985113.
[I 2024-05-23 01:07:25,388] Trial 17 finished with value: 0.9643306629998344 and parameters: {'learning_rate': 0.03319278550826355, 'l1_coef': 7.475199555377638e-07, 'patience': 10, 'batch_size': 64, 'lr_factor': 0.46163221633999585, 'num_epochs': 161}. Best is trial 2 with value: 0.7424067212985113.
[I 2024-05-23 01:08:26,448] Trial 14 finished with value: 1.814862275123596 and parameters: {'learning_rate': 0.02812313145521612, 'l1_coef': 0.03017157290208268, 'patience': 14

Chr 2 - Best hyperparameters: {'learning_rate': 0.001813751389499825, 'l1_coef': 7.793207352687452e-08, 'patience': 9, 'batch_size': 32, 'lr_factor': 0.6069847013701387, 'num_epochs': 423}
Chr 2 - Best value: 0.6719
Final model saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/models_unphased/chr2/final_model_chr2.pth
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr2/individual_r2_scores_chr2.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr2/individual_iqs_scores_chr2.csv
Performance metrics saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/performance_metrics.csv
Total SNPs:  963
PRS313 SNPs:  16
Total SNPs used for Training:  947


...

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_98377/4219423060.py:25: RuntimeWarning: invalid value encountered in divide
  chi_squared = np.sum((contingency_table - expected_counts) ** 2 / expected_counts)
[I 2024-05-23 11:47:18,628] A new study created in RDB with name: unphased_full_23andMe_chr21_study_logistic_regression
[I 2024-05-23 11:48:18,055] Trial 6 finished with value: 0.7160479247570037 and parameters: {'learning_rate': 0.045724692439618665, 'l1_coef': 7.881446682339455e-09, 'patience': 15, 'batch_size': 256, 'lr_factor': 0.30953500093983594, 'num_epochs': 132}. Best is trial 6 with value: 0.7160479247570037.
[I 2024-05-23 11:49:27,296] Trial 5 finished with value: 0.7142757599170391 and parameters: {'learning_rate': 0.02669869306219022, 'l1_coef': 8.604820920110499e-08, 'patience': 16, 'batch_size': 32, 'lr_factor': 0.645036029775653, 'num_epochs': 262}. Best is trial 5 with value: 0.7142757599170391.
[I 2024-05-23 11:52:00,179] Trial 3 finished with value: 0

Chr 21 - Best hyperparameters: {'learning_rate': 0.06417463659123403, 'l1_coef': 1.4994401968392755e-08, 'patience': 20, 'batch_size': 64, 'lr_factor': 0.8953661415766634, 'num_epochs': 117}
Chr 21 - Best value: 0.6684
Final model saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/models_unphased/chr21/final_model_chr21.pth
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr21/individual_r2_scores_chr21.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr21/individual_iqs_scores_chr21.csv
Performance metrics saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/performance_metrics.csv
Total SNPs:  443
PRS313 SNPs:  11
Total SNPs used for Training:  432


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_98377/4219423060.py:25: RuntimeWarning: invalid value encountered in divide
  chi_squared = np.sum((contingency_table - expected_counts) ** 2 / expected_counts)
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_98377/4219423060.py:25: RuntimeWarning: invalid value encountered in divide
  chi_squared = np.sum((contingency_table - expected_counts) ** 2 / expected_counts)
[I 2024-05-23 12:21:01,776] Trial 22 finished with value: 0.9262316722136278 and parameters: {'learning_rate': 0.05079781889732127, 'l1_coef': 0.006007062333265984, 'patience': 10, 'batch_size': 32, 'lr_factor': 0.1757661398292724, 'num_epochs': 306}. Best is trial 3 with value: 0.7342786401510238.
[I 2024-05-23 12:21:10,050] Trial 23 finished with value: 0.696631726196834 and parameters: {'learning_rate': 0.0004549303724999769, 'l1_coef': 3.038022393227062e-06, 'patience': 9, 'batch_size': 64, 'lr_factor': 0.6520813926376375, 'num_epochs': 102}. Best is

Chr 22 - Best hyperparameters: {'learning_rate': 0.0008842867535788688, 'l1_coef': 3.953662605180574e-10, 'patience': 15, 'batch_size': 32, 'lr_factor': 0.7809338408689392, 'num_epochs': 364}
Chr 22 - Best value: 0.6481
Final model saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/models_unphased/chr22/final_model_chr22.pth
Individual R^2 scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr22/individual_r2_scores_chr22.csv
Individual IQS scores saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/chr22/individual_iqs_scores_chr22.csv
Performance metrics saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/performance_metrics.csv


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_98377/4219423060.py:25: RuntimeWarning: invalid value encountered in divide
  chi_squared = np.sum((contingency_table - expected_counts) ** 2 / expected_counts)


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as sklearn_r2_score, accuracy_score
import os
import csv

# Load and preprocess the data
data_directory = '../../Data/Filtered_unphased_training_data_union_final/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
r2_scores = []
iqs_scores = []
accuracy_scores = []

# Folders for saved models and CSV files
output_folder = "../../Data/model_results_unphased_all_PRS/logistic_regression/"
model_folder = output_folder + "models_unphased/"
csv_folder = output_folder + "csv_files/"

def calculate_iqs_unphased(y_true, y_pred):
    # Dummy implementation of IQS calculation, replace with the actual function
    return np.mean(np.abs(y_true - y_pred))

for chromosome_number in range(start, 23):
    # Paths for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*PRS313_)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='PRS313_').values, dtype=torch.long)

    print("Total SNPs: ", data.shape[1])
    print("PRS313 SNPs: ", y.shape[1])
    print("Total SNPs used for Training: ", X.shape[1])

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the multinomial logistic regression model with L1 regularization
    class MulticlassLogisticRegression(nn.Module):
        def __init__(self, input_dim, output_dim, num_classes=3, l1_coef=0.0):
            super(MulticlassLogisticRegression, self).__init__()
            self.num_classes = num_classes
            self.linear = nn.ModuleList([nn.Linear(input_dim, output_dim) for _ in range(num_classes)])
            self.l1_coef = l1_coef

        def forward(self, x):
            out = torch.stack([linear(x) for linear in self.linear], dim=-1)
            out = nn.functional.softmax(out, dim=-1)
            return out

        def l1_loss(self):
            return self.l1_coef * sum(torch.norm(linear.weight, p=1) for linear in self.linear)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the final model
    model = MulticlassLogisticRegression(X_train_val.shape[1], y_train_val.shape[1])
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model.to(device)
    model.eval()

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_outputs = test_outputs.argmax(dim=-1)
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_iqs = calculate_iqs_unphased(y_test.cpu().numpy(), test_outputs.cpu().numpy())
        test_accuracy = ((y_test == test_outputs).sum() / (test_outputs.shape[0] * test_outputs.shape[1])).numpy()

        # Append performance metrics to the lists
        r2_scores.append(test_r2)
        iqs_scores.append(test_iqs)
        accuracy_scores.append(test_accuracy)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Calculate individual IQS scores for each SNP
        individual_iqs_scores = np.array([calculate_iqs_unphased(y_test.cpu().numpy()[:, i].reshape(-1, 1), test_outputs.cpu().numpy()[:, i].reshape(-1, 1)) for i in range(y_test.shape[1])])

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='PRS').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual IQS scores to a CSV file
        iqs_csv_file = chr_csv_folder + f'individual_iqs_scores_chr{chromosome_number}.csv'

        with open(iqs_csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'IQS Score'])
            for snp, iqs_score in zip(snp_names, individual_iqs_scores):
                writer.writerow([snp, iqs_score])

        print(f"Individual IQS scores saved at: {iqs_csv_file}")

# Create a DataFrame to store the performance metrics for each chromosome
performance_df = pd.DataFrame({
    'Chromosome': list(range(start, 23)),
    'R2 Score': r2_scores,
    'IQS Score': iqs_scores,
    'Accuracy Score': accuracy_scores
})

# Save the performance metrics to a CSV file
performance_csv_file = csv_folder + 'performance_metrics.csv'
performance_df.to_csv(performance_csv_file, index=False)
print(f"Performance metrics saved at: {performance_csv_file}")


In [25]:
# Loop through all the training datasets and document the PRS313 SNPs in each dataset. Save this to a CSV file.

import pandas as pd
import os

data_directory = '../../Data/Filtered_unphased_training_data_union_final/'
output_folder = "../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Initialize a list to store the PRS313 SNPs in each dataset
prs313_snps = []

for chromosome_number in range(1, 23):
    file_name = data_directory + \
        f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_combined.parquet"
    data = pd.read_parquet(file_name)

    prs313_snps.append(data.filter(regex='PRS313_').columns)

# Create a DataFrame to store the PRS313 SNPs in each dataset
prs313_df = pd.DataFrame({
    'Chromosome': list(range(1, 23)),
    'PRS313 SNPs': prs313_snps,
    "Number of PRS313 SNPs": [len(snps) for snps in prs313_snps]
})

# Save the PRS313 SNPs to a CSV file
prs313_csv_file = output_folder + 'prs313_snps.csv'
prs313_df.to_csv(prs313_csv_file, index=False)
print(f"PRS313 SNPs saved at: {prs313_csv_file}")

# Print the total number of PRS313 SNPs in all datasets
total_prs313_snps = sum(prs313_df["Number of PRS313 SNPs"])
print(f"Total number of PRS313 SNPs: {total_prs313_snps}")



PRS313 SNPs saved at: ../../Data/model_results_unphased_all_PRS/logistic_regression/csv_files/prs313_snps.csv
Total number of PRS313 SNPs: 313
